# Are We Rich Yet?

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [3]:
def calculate_commission(price, quantity):
    #fix rate pricing from IB including all exchange and regularotry fees
    #https://www.interactivebrokers.com/en/index.php?f=1590&p=stocks1&ns=T
    return np.minimum(np.maximum(0.005*quantity, 1), price*quantity*0.005)

'''
input:
signal_column: name of the column that indicates the signal,  1 for long signal or -1 for short signal
price_column: name of the column responding to order execution price, could be open or close
include_commision: deduct commission for each transaction if True, default to True

output: 
balance, profit, ROI

NOTE: this is LONG only evaluation, SHORT is not implemented yet. SHORT position will involve margin, so we need to decide 
later.

'''
# Long only
def evaluate_profit(df, intitial_balance, signal_column, price_column, include_commission=True):

    balance = intitial_balance
    current_position = 0 #1 for long position, -1 for short, 0 for closed position or no position
    pos_quantity = 0 # current position quantity
    pos_price = 0 #current position price

    #iterate from first to n-1 row, as we want to close our position on the last day.
    for index, row in df.head(-1).iterrows():
        new_position = row[signal_column]
        current_price = row[price_column]
        
        # receive signal to reverse current position
        if current_position != new_position:
            # bullish market: close short position and open a long position
            if new_position == 1:
                #open a long position (short position not implemented yet)
                pos_quantity = np.floor_divide(balance, current_price) 
                balance = balance - current_price * pos_quantity
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)
                
                pos_price = current_price
                current_position = new_position
                #print "long ", row['date'], balance, pos_quantity, pos_price
            
            # bearish market: close long position and open a short position
            else:
                #close the long position (short position not implemented yet)
                balance = balance + current_price * pos_quantity
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)
                    
                pos_quantity = 0
                pos_price = 0
                current_position = new_position
                #print "close", row['date'], balance, pos_quantity, pos_price
            
        #hold position and do nothing
        else:
            #print "hold ", row['date'], balance, pos_quantity, pos_price
            pass
            
            
    #found open position on the last day, let's close it
    if pos_quantity != 0:
        # get price of last day
        current_price = df.tail(1).iloc[0][price_column]
        balance = balance + pos_quantity * current_price
        if include_commission:
            balance = balance - calculate_commission(current_price, pos_quantity)
    
    profit = balance - intitial_balance
    ROI = profit/intitial_balance
    
    return  balance, profit, ROI

# Long and Short
def evaluate_profit2(df, intitial_balance, signal_column, price_column, include_commission=True):

    balance = intitial_balance
    current_position = -1 #1 for long position, 0 for short
    pos_quantity = 0 # current position quantity
    pos_price = 0 #current position price

    #iterate from first to n-1 row, as we want to close our position on the last day.
    for index, row in df.head(-1).iterrows():
        new_position = row[signal_column]
        current_price = row[price_column]
        
        # receive signal to reverse current position
        if current_position != new_position:
            # bullish market: close short position and open a long position
            if new_position == 1:
                #close the short position
                balance = balance + pos_quantity * (2 * pos_price - current_price)
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)

                #open a long position 
                pos_quantity = np.floor_divide(balance, current_price) 
                balance = balance - current_price * pos_quantity
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)
                
                pos_price = current_price
                current_position = new_position
                #print "long ", row['date'], balance, pos_quantity, pos_price
            
            # bearish market: close long position and open a short position
            else:
                #close the long position
                balance = balance + current_price * pos_quantity
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)
                
                #open a short position
                pos_quantity = np.floor_divide(balance, current_price) 
                balance = balance - current_price * pos_quantity
                pos_price = current_price
                current_position = new_position
                #print "close", row['date'], balance, pos_quantity, pos_price
            
        #hold position and do nothing
        else:
            #print "hold ", row['date'], balance, pos_quantity, pos_price
            pass
            
            
    #found open position on the last day, let's close it
    if pos_quantity != 0:
        # get price of last day
        current_price = df.tail(1).iloc[0][price_column]
        
        #close long position
        if current_position == 1:
            balance = balance + pos_quantity * current_price
            if include_commission:
                balance = balance - calculate_commission(current_price, pos_quantity)
        
        #close short position
        else:
            balance = balance + pos_quantity * (2 * pos_price - current_price)
            if include_commission:
                balance = balance - calculate_commission(current_price, pos_quantity)
            
    profit = balance - intitial_balance
    ROI = profit/intitial_balance
    
    return  balance, profit, ROI

### Get data

In [4]:
df=pd.read_csv("data/IYR.csv")
df.head()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
0,2010-01-04,46.450001,46.660000,45.529999,45.810001,16952700,45.919998,True,0.002401,43.799999,False,-0.043877,1,47.639866,45.7345,43.829134,0.519813,8.332290,False,False,False,0,44.250796,40.977224,40.666430,1,1,42.964314,41.446505,39.687018,1,1,47.899399,0,29.532814,25.622815,19.705172,True,True,1,80,1,-19.985297,1,0.780138,0.868801,-0.088663,0,2.428798,1,56.828598,0,1.215208,0,53.874570,0,41.244019,0,-63.392854,0,-3233039.650530,0,16.105949,29.281134,1,0.000000,2.298156,1,0.189764,1,46.000000,50.419998,44.060001,52.359997,39.640003,56.779995,37.700004,1
1,2010-01-05,45.810001,45.959999,45.290001,45.919998,14523000,45.900002,False,-0.000435,44.119999,False,-0.039199,1,47.660512,45.7585,43.856488,0.542455,8.313262,False,False,False,0,44.316255,41.042691,40.718704,1,1,43.012126,41.525376,39.812894,1,1,47.804623,0,28.008132,24.242509,20.573319,True,True,1,80,1,-53.292530,1,0.703839,0.835809,-0.131969,0,2.249153,1,48.760069,0,0.679669,0,54.661907,0,43.073801,0,-65.584491,0,-657379.254835,0,12.573499,17.236026,1,8.977150,2.992383,1,0.191900,1,45.723333,49.866663,43.506666,52.083330,39.363336,56.226660,37.146669,1
2,2010-01-06,45.939999,46.230000,45.709999,45.900002,14859100,46.310001,True,0.008932,43.740002,False,-0.047059,1,47.654105,45.8210,43.987895,0.521549,8.001158,False,False,False,0,44.378363,41.107026,40.770260,1,1,43.066403,41.602976,39.916297,1,1,47.653746,0,26.917506,25.435159,19.686896,True,True,1,76,1,-37.012487,1,0.634445,0.795536,-0.161091,0,2.016278,1,56.429146,0,-0.563254,0,54.479851,0,39.780550,0,-66.233712,0,-816785.273366,0,22.999634,17.226361,1,6.901372,5.292841,1,0.193347,1,45.946667,50.313332,43.953335,52.306664,39.586670,56.673329,37.593338,1
3,2010-01-07,45.869999,46.520000,45.369999,46.310001,16364700,46.000000,False,-0.006694,44.139999,False,-0.046858,0,47.636673,45.9175,44.198327,0.614154,7.488094,False,False,False,0,44.454113,41.175940,40.825381,1,1,43.139411,41.682390,40.041486,1,1,47.511921,0,25.397447,23.068010,20.606229,True,True,1,72,1,-29.312112,1,0.605552,0.757539,-0.151987,0,1.895537,1,55.764453,0,-0.728834,0,57.598215,0,46.390587,0,-52.922047,0,2497548.393800,1,36.835206,24.136113,1,100.000000,38.626174,0,0.194417,1,46.066667,50.553331,44.193334,52.426664,39.706670,56.913328,37.833337,1
4,2010-01-08,46.189999,46.230000,45.590000,46.000000,10629500,46.220001,True,0.004783,43.720001,False,-0.049565,1,47.556586,46.0015,44.446414,0.499518,6.761021,False,False,False,0,44.514736,41.239835,40.876870,1,1,43.210449,41.756302,40.157529,1,1,47.378606,0,23.985964,21.705970,19.389544,True,True,1,68,1,-56.291656,1,0.551285,0.716289,-0.165003,0,1.693048,1,57.228793,0,-1.435608,0,54.555067,0,51.744576,0,-62.987021,0,4577581.772970,1,45.573585,35.136142,1,18.275015,41.725462,1,0.194916,1,45.940000,50.299998,43.940001,52.299997,39.580003,56.659995,37.580004,1


In [5]:
df.tail()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
1478,2015-11-16,72.300003,73.309998,72.300003,73.290001,5939900,73.330002,True,0.000546,61.07,False,-0.166735,1,77.820998,75.097000,72.373003,0.168318,7.254610,False,False,False,1,73.955311,74.243210,74.345325,0,0,73.759056,74.300903,75.396076,0,0,76.061274,0,26.601219,13.698139,29.176343,True,False,0,-40,0,-88.983026,1,-0.212274,0.235812,-0.448086,0,-0.639207,0,42.565106,0,-4.830543,1,44.321554,1,42.429896,0,-79.113957,1,-2735546.954670,0,27.219841,25.850562,1,86.823377,42.999771,0,0.058669,1,72.966667,73.633332,68.893334,77.706665,68.226669,78.373330,64.153336,0
1479,2015-11-17,73.190002,73.959999,72.959999,73.330002,9096300,74.110001,True,0.010637,59.48,False,-0.188872,1,77.753154,74.961500,72.169847,0.207790,7.448232,False,False,False,1,73.930789,74.231115,74.335222,0,0,73.750932,74.291788,75.372379,0,0,75.760373,0,26.317597,17.164157,27.205156,True,False,0,-40,0,-57.559205,1,-0.254883,0.137673,-0.392556,0,-0.800961,0,39.684043,0,-3.905119,1,44.580679,1,45.184818,0,-78.270054,1,-3147116.232380,0,38.029741,28.505516,1,90.799039,59.207472,0,0.056971,1,73.416667,74.533330,69.793332,78.156665,68.676669,79.273328,65.053334,0
1480,2015-11-18,73.589996,74.169998,73.120003,74.110001,8293200,74.320000,True,0.002834,60.25,False,-0.187019,1,77.667774,74.878501,72.089227,0.362240,7.450132,False,False,False,1,73.937817,74.229511,74.332981,0,0,73.761425,74.290368,75.359310,0,0,75.483543,0,25.762636,17.365427,25.274308,True,False,0,-40,0,-32.375396,1,-0.223140,0.065511,-0.288650,0,-1.123941,0,39.859897,0,-2.409798,1,49.514647,1,52.027057,0,-61.814372,1,-692195.818664,0,70.091546,45.113709,1,100.000000,92.540805,0,0.055227,1,73.800001,75.299998,70.560000,78.539999,69.060003,80.039996,65.820002,0
1481,2015-11-19,74.279999,74.620003,74.019997,74.320000,6536100,75.080002,True,0.010226,59.52,False,-0.199139,1,77.447361,74.764501,72.081641,0.417159,7.176829,False,False,False,1,73.952804,74.230709,74.332852,1,0,73.778100,74.290596,75.347808,1,0,75.228860,0,24.643201,19.777373,24.216607,True,False,0,-40,0,1.377413,1,-0.178975,0.016614,-0.195588,0,-1.380651,0,45.259109,0,-2.415972,1,50.785046,1,49.923065,0,-57.384012,1,375016.697735,1,80.575192,62.898826,0,100.000000,96.933013,0,0.053529,1,74.320000,76.339997,71.599999,79.059998,69.580002,81.079995,66.860001,0
1482,2015-11-20,74.389999,75.169998,74.379997,75.080002,7804800,65.150002,False,-0.132259,59.02,False,-0.213905,0,77.361753,74.724001,72.086249,0.567482,7.059986,False,False,False,1,73.997008,74.241958,74.340286,1,0,73.814288,74.295965,75.345608,1,0,72.300003,1,22.928698,22.473685,22.763272,True,False,0,-40,0,48.016317,1,-0.081706,-0.003050,-0.078656,0,-1.506564,0,39.645890,0,1.500610,1,55.180686,1,52.274667,0,-37.807545,1,2695056.981140,1,93.574892,81.413877,0,100.000000,100.000000,0,0.052153,1,74.876666,77.453328,72.713330,79.616664,70.136668,82.193326,67.973332,0


In [6]:
dftouse=df.copy()

### Feature Engineering

In [7]:
IGNORE = ['date', 'result_1','close_1','perf_1','result_14','close_14','perf_14','results']

In [8]:
INDICATORS=[]
for v in df.columns:
    l=df[v].unique()
    if len(l) <= 10 and v not in IGNORE:
        print v, l
        INDICATORS.append(v)

bb_squeeze [False True]
bb_signalup [False True]
bb_signaldn [False True]
bb_signal [ 0.  1.]
ema_signal1 [1 0]
ema_signal2 [1 0]
kama_signal1 [1 0]
kama_signal2 [1 0]
sar_signal [0 1]
adx_trend [True False]
adx_direction [True False]
adx_signal [ 1.  0.]
aroon_signal [1 0]
cci_signal [ 1.  0.]
macd_signal [0 1]
ppo_signal [1 0]
mfi_signal [ 0.  1.]
roc_signal [ 0.  1.]
rsi_signal [ 0.  1.]
ult_signal [ 0.  1.]
wr_signal [ 0.  1.]
ad_signal [0 1]
sslow_signal [ 1.  0.]
srsi_signal [ 1.  0.]
trix_signal [1 0]
cv_signal [1 0]


In [9]:
STANDARDIZABLE = []
for v in df.columns:
    if v not in INDICATORS and v not in IGNORE:
        print v
        STANDARDIZABLE.append(v)

open
high
low
close
volume
bb_upper
bb_middle
bb_lower
bb_pct
bb_bandwidth
ema50
ema150
ema200
kama50
kama150
kama200
sar
adx
plus_di
minus_di
aroon_osc
cci
macd
macd_sigline
macd_hist
ppo
mfi
roc
rsi
ult_osc
willr
ad_osc
stoch_slowk
stoch_slowd
stoch_fastk
stoch_fastd
trix
sr_pivotpts
sr_res1
sr_sup1
sr_res2
sr_sup2
sr_res3
sr_sup3


In [10]:
# from sklearn.cross_validation import train_test_split
# itrain, itest = train_test_split(xrange(dftouse.shape[0]), train_size=0.7)
# mask=np.ones(dftouse.shape[0], dtype='int')
# mask[itrain]=1
# mask[itest]=0
# mask = (mask==1)
# mask.shape, mask.sum()

In [11]:
dftouse['date'] = pd.to_datetime(dftouse['date'])
mask = (dftouse.date < '2015-01-01').values
mask.shape, mask.sum()

((1483,), 1258)

#### 1.2 Standardize the data

Use the mask to compute the training and test parts of the dataframe. Use `StandardScaler` from `sklearn.preprocessing` to "fit" the columns in `STANDARDIZABLE` on the training set. Then use the resultant estimator to transform both the training and the test parts of each of the columns in the dataframe, replacing the old unstandardized values in the `STANDARDIZABLE` columns of `dftouse` by the new standardized ones.

In [12]:
#your code here
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(dftouse[mask][STANDARDIZABLE])
dftouse[STANDARDIZABLE] = scaler.transform(dftouse[STANDARDIZABLE])
dftouse.head()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
0,2010-01-04,-1.974789,-2.021873,-2.010345,-2.057763,1.257904,45.919998,True,0.002401,43.799999,False,-0.043877,1,-2.150531,-2.061784,-1.942517,-0.223879,0.305631,False,False,False,0,-2.219221,-2.371982,-2.298187,1,1,-2.355040,-2.339656,-2.535415,1,1,-1.692185,0,0.578886,0.425673,-0.134153,True,True,1,0.961322,1,-0.422823,1,0.964501,1.204992,-0.391073,0,1.286500,1,0.260301,0,0.219601,0,-0.037623,0,-1.324320,0,-0.771631,0,-0.859011,0,-1.575159,-1.161855,1,-1.275851,-1.530970,1,1.620204,1,-2.031051,-1.927240,-1.791896,-2.103268,-1.826986,-1.893877,-1.599763,1
1,2010-01-05,-2.056848,-2.112559,-2.040752,-2.043666,0.807210,45.900002,False,-0.000435,44.119999,False,-0.039199,1,-2.147773,-2.058671,-1.939132,-0.154135,0.300964,False,False,False,0,-2.210674,-2.363794,-2.291755,1,1,-2.348911,-2.329997,-2.520053,1,1,-1.703985,0,0.391414,0.192819,-0.023553,True,True,1,0.961322,1,-0.737174,1,0.842441,1.147439,-0.583850,0,1.178123,1,-0.236236,0,0.061919,0,0.031994,0,-1.148810,0,-0.843764,0,-0.517596,0,-1.709058,-1.654943,1,-1.062164,-1.510250,1,1.644016,1,-2.066518,-1.999432,-1.855567,-2.141951,-1.857354,-1.968885,-1.653827,1
2,2010-01-06,-2.040180,-2.077580,-1.987539,-2.046229,0.869554,46.310001,True,0.008932,43.740002,False,-0.047059,1,-2.148629,-2.050565,-1.922867,-0.218532,0.224415,False,False,False,0,-2.202566,-2.355748,-2.285410,1,1,-2.341953,-2.320493,-2.507433,1,1,-1.722771,0,0.257313,0.394016,-0.136481,True,True,1,0.894666,1,-0.583524,1,0.731427,1.077185,-0.713485,0,1.037635,1,0.235718,0,-0.304044,0,0.015896,0,-1.464693,0,-0.865131,0,-0.538726,0,-1.313851,-1.655338,1,-1.111575,-1.441590,1,1.660148,1,-2.037888,-1.941157,-1.804170,-2.110725,-1.832840,-1.908336,-1.610185,1
3,2010-01-07,-2.049155,-2.040010,-2.030616,-1.993686,1.148834,46.000000,False,-0.006694,44.139999,False,-0.046858,0,-2.150957,-2.038049,-1.896822,0.066723,0.098578,False,False,False,0,-2.192676,-2.347129,-2.278627,1,1,-2.332593,-2.310767,-2.492155,1,1,-1.740430,0,0.070410,-0.005315,-0.019360,True,True,1,0.828010,1,-0.510848,1,0.685206,1.010902,-0.672958,0,0.964794,1,0.194813,0,-0.352797,0,0.291625,0,-0.830668,0,-0.427009,0,-0.099395,1,-0.789409,-1.372476,1,1.104482,-0.446710,0,1.672075,1,-2.022505,-1.909845,-1.776554,-2.093947,-1.819668,-1.875803,-1.586736,1
4,2010-01-08,-2.008126,-2.077580,-2.002743,-2.033414,0.084989,46.220001,True,0.004783,43.720001,False,-0.049565,1,-2.161656,-2.027154,-1.866116,-0.286395,-0.079749,False,False,False,0,-2.184761,-2.339137,-2.272291,1,1,-2.323487,-2.301715,-2.477993,1,1,-1.757029,0,-0.103143,-0.235088,-0.174363,True,True,1,0.761354,1,-0.765480,1,0.598393,0.938942,-0.730899,0,0.842637,1,0.284929,0,-0.560898,0,0.022547,0,-0.317122,0,-0.758274,0,0.176323,1,-0.458178,-0.922170,1,-0.840844,-0.354208,1,1.677631,1,-2.038743,-1.942896,-1.805704,-2.111657,-1.833572,-1.910144,-1.611488,1


We create a list `lcols` of the columns we will use in our classifier. This list should not contain the response `RESP`. How many features do we have?

In [13]:
#lcols=list(dftouse.columns)
#lcols.remove(u'results')
lcols=[]
for c in list(dftouse.columns):
    if c not in IGNORE:
        lcols.append(c)
print len(lcols)

70


### EDA for the data

We create a variable `ccols` which contains all variables not in our indicators list

In [14]:
ccols=[]
for c in lcols:
    if c not in INDICATORS and c not in IGNORE:
        ccols.append(c)
print len(ccols), len(INDICATORS)
ccols

44 26


['open',
 'high',
 'low',
 'close',
 'volume',
 'bb_upper',
 'bb_middle',
 'bb_lower',
 'bb_pct',
 'bb_bandwidth',
 'ema50',
 'ema150',
 'ema200',
 'kama50',
 'kama150',
 'kama200',
 'sar',
 'adx',
 'plus_di',
 'minus_di',
 'aroon_osc',
 'cci',
 'macd',
 'macd_sigline',
 'macd_hist',
 'ppo',
 'mfi',
 'roc',
 'rsi',
 'ult_osc',
 'willr',
 'ad_osc',
 'stoch_slowk',
 'stoch_slowd',
 'stoch_fastk',
 'stoch_fastd',
 'trix',
 'sr_pivotpts',
 'sr_res1',
 'sr_sup1',
 'sr_res2',
 'sr_sup2',
 'sr_res3',
 'sr_sup3']

#### 1.4 Train a SVM on this data.

In [15]:
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV

In [16]:
"""
Function
--------
cv_optimize

Inputs
------
clf : an instance of a scikit-learn classifier
parameters: a parameter grid dictionary thats passed to GridSearchCV (see above)
X: a samples-features matrix in the scikit-learn style
y: the response vectors of 1s and 0s (+ives and -ives)
n_folds: the number of cross-validation folds (default 5)
score_func: a score function we might want to pass (default python None)
   
Returns
-------
The best estimator from the GridSearchCV, after the GridSearchCV has been used to
fit the model.
     
Notes
-----
see do_classify and the code below for an example of how this is used
"""
#your code here
def cv_optimize(clf, parameters, X, y, n_folds, score_func):
    fitmodel = GridSearchCV(clf, param_grid=parameters, cv=n_folds, scoring=score_func)
    fitmodel.fit(X, y)
    return fitmodel.best_estimator_

In [86]:
from sklearn.metrics import confusion_matrix
def do_classify(clf, parameters, indf, featurenames, targetname, target1val, mask=None, reuse_split=None, score_func=None, n_folds=5):
    subdf=indf[featurenames]
    X=subdf.values
    y=(indf[targetname].values==target1val)*1
    if mask !=None:
        #print "using mask"
        Xtrain, Xtest, ytrain, ytest = X[mask], X[~mask], y[mask], y[~mask]
    if reuse_split !=None:
        #print "using reuse split"
        Xtrain, Xtest, ytrain, ytest = reuse_split['Xtrain'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['ytest']
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    #print "############# based on standard predict ################"
    print "Accuracy on training data: %0.2f" % (training_accuracy)
    print "Accuracy on test data:     %0.2f" % (test_accuracy)
    #print confusion_matrix(ytest, clf.predict(Xtest))
    print "########################################################"
    return clf, Xtrain, ytrain, Xtest, ytest

In [57]:
%%time
clfsvm, Xtrain, ytrain, Xtest, ytest = do_classify(LinearSVC(loss="hinge"), {"C": [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]}, dftouse, lcols, u'results',1, mask=mask)

############# based on standard predict ################
Accuracy on training data: 0.70
Accuracy on test data:     0.72
########################################################
CPU times: user 1.59 s, sys: 32.1 ms, total: 1.63 s
Wall time: 1.7 s


/Users/Yvan-Laptop/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [58]:
clfsvm

LinearSVC(C=0.001, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0)

In [59]:
#your code here

def evaluate(clf):

    ypred = clf.predict(Xtest)
    df_pred = df[~mask].reset_index(drop=True)
    df_pred['pred_result'] = ypred
    df_pred['result_baseline'] = np.ones(df_pred.shape[0])
    #print "accuracy on test set: {0:.3f}".format((df_pred.results == df_pred.pred_result).sum()/float(len(df_pred)))
    
    balance, profit, ROI = evaluate_profit2(df_pred, 10000, 'result_baseline', 'close', False)
    print "ROI baseline: {0:.2f}%".format(ROI*100)
    
    balance, profit, ROI = evaluate_profit(df_pred, 10000, 'results', 'close', False)
    print 'ROI "result" buy-only: {0:.2f}%'.format(ROI*100)
    
    balance, profit, ROI = evaluate_profit2(df_pred, 10000, 'results', 'close', False)
    print 'ROI "result" buy-sell: {0:.2f}%'.format(ROI*100)
    
    balance, profit, ROI = evaluate_profit(df_pred, 10000, 'pred_result', 'close', False)
    print 'ROI "pred" buy-only: {0:.2f}%'.format(ROI*100)
    
    balance, profit, ROI = evaluate_profit2(df_pred, 10000, 'pred_result', 'close', False)
    print 'ROI "pred" buy-sell: {0:.2f}%'.format(ROI*100)
    
evaluate(clfsvm)

ROI baseline: -3.47%
ROI "result" buy-only: 50.29%
ROI "result" buy-sell: 130.60%
ROI "pred" buy-only: -2.87%
ROI "pred" buy-sell: -2.82%


The results obtained ought to be very similar to the efforts you put in earlier. If not its likely you wrote `cv_optimize` wrong. (Remember that we are using the same mask).

We'll reuse the training and test sets you computed above later in the homework. We do this by putting them into a dictionary `reuse_split`

In [60]:
reuse_split=dict(Xtrain=Xtrain, Xtest=Xtest, ytrain=ytrain, ytest=ytest)

## 2. Estimate costs and benefits from assumptions and data

### Our data is highly asymmetric

First notice that our data set is very highly asymmetric, with positive `RESP`onses only making up 16-17% of the samples.

In [61]:
print "whole data set", dftouse['results'].mean()#Highly asymmetric
print "training set", dftouse['results'][mask].mean(), "test set", dftouse['results'][~mask].mean()

whole data set 0.458530006743
training set 0.452305246423 test set 0.493333333333


This means that a classifier which predicts that EVERY customer is a negative has an accuracy rate of 83-84%. By this we mean that **a classifier that predicts that no customer will respond to our mailing** has an accuracy of 83-84%!

In [62]:
#your code here
from sklearn.linear_model import LogisticRegression
clflog = LogisticRegression(penalty="l1")
parameters = {"C": [0.001, 0.01, 0.1, 1, 10, 100]}
clflog, Xtrain, ytrain, Xtest, ytest = do_classify(clflog, parameters, dftouse, lcols, u'results',1, mask=mask, reuse_split=reuse_split)

############# based on standard predict ################
Accuracy on training data: 0.73
Accuracy on test data:     0.65
########################################################


/Users/Yvan-Laptop/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [63]:
clflog

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [64]:
evaluate(clflog)

ROI baseline: -3.47%
ROI "result" buy-only: 50.29%
ROI "result" buy-sell: 130.60%
ROI "pred" buy-only: -6.96%
ROI "pred" buy-sell: -11.19%


## 4. Trying to improve the SVM: Feature Selection and Data Balancing

If you did the previous section right, you will find that the logistic regression model provides a better profit over some section of the profit curve than the baseline "send to everyone" classifier, while the SVM classifier is generally poor. At this might we might want to try all kinds of classifiers: from perceptrons to random forests. In the interest of time, and to study the SVM in some more detail, we'll restrict ourselves to trying to improve the SVM performance here. In real life you would try other classifiers as well.

 We wont be exhaustive in this improvement process either(which is something you should do on your project) in the interests of time, but we'll explore if feature-selection on the  SVM, and data balancing on the SVM (SVM's are known to perform better on balanced data) help.
 
( An aside: many classifiers such as SVM and decision trees struggle in their techniques on imbalanced data. You can read more at: see Weiss, Gary M., and Foster Provost. "The effect of class distribution on classifier learning: an empirical study." Rutgers Univ (2001). Also see http://pages.stern.nyu.edu/~fprovost/Papers/skew.PDF and http://www.cs.ox.ac.uk/people/vasile.palade/papers/Class-Imbalance-SVM.pdf for multiple ways to deal with the imbalance problem: balancing is not always the best option. `Sklearn` also provides a class weighting strategy: http://scikit-learn.org/stable/modules/svm.html#unbalanced-problems ). 

### Feature Selection

The Lasso, for example, implements internally, a form of feature selection by setting many coefficients to zero. Let us find coefficients that are non-zero.

#### Non zero lasso features

We write a function `nonzero_lasso` which takes the fit classifier `clfloglasso` as an argument, and spits out a dataframe of coefficients, sorted by the absolute magnitude of the coefficients. This way we can see which features dominated the logistic regression.

In [65]:
def nonzero_lasso(clf):
    featuremask=(clf.coef_ !=0.0)[0]
    return pd.DataFrame(dict(feature=lcols, coef=clf.coef_[0], abscoef=np.abs(clf.coef_[0])))[featuremask].sort('abscoef', ascending=False)

In [66]:
lasso_importances=nonzero_lasso(clflog)
lasso_importances.set_index("feature", inplace=True)
lasso_importances.head(10)

,abscoef,coef
feature,,
cv_signal,0.522058,0.522058
plus_di,0.407759,-0.407759
stoch_fastk,0.351714,-0.351714
stoch_fastd,0.272643,-0.272643
ad_signal,0.239186,-0.239186
ad_osc,0.174617,-0.174617
cci_signal,0.173283,-0.173283
wr_signal,0.164776,-0.164776
bb_bandwidth,0.147666,-0.147666


#### 4.1 Feature importance using correlations

We can also get a notion of which features are important in the classification process by seeing how they correlate with the response. Implement some code to obtain the Pearson correlation coefficient between each of our features and the response. Do this on the training set only! Create a dataframe indexed by the features, which has columns `abscorr` the absolute value of the correlation and `corr` the value of the correlation. Sort the dataframe by `abscorr`, highest first, and show the top 25 features with the highest absolute correlation. Is there much overlap with the feature selection performed by the LASSO?

In [67]:
from scipy.stats.stats import pearsonr
correlations=[]
dftousetrain=dftouse[mask]
for col in lcols:
    r=pearsonr(dftousetrain[col], dftousetrain['results'])[0]
    pvals=pearsonr(dftousetrain[col], dftousetrain['results'])[1]
    correlations.append(dict(feature=col,corr=r, abscorr=np.abs(r), pvals=pvals))

bpdf=pd.DataFrame(correlations).sort('abscorr', ascending=False)
bpdf.set_index(['feature'], inplace=True)
bpdf.head(25)

,abscorr,corr,pvals
feature,,,
stoch_slowk,0.402160,-0.402160,4.270360e-50
stoch_fastd,0.372846,-0.372846,9.041199e-43
willr,0.370550,-0.370550,3.154957e-42
bb_pct,0.364357,-0.364357,8.738056e-41
rsi,0.364304,-0.364304,8.987875e-41
cci,0.361771,-0.361771,3.421182e-40
ult_osc,0.361392,-0.361392,4.174971e-40
stoch_slowd,0.360162,-0.360162,7.951304e-40
plus_di,0.351510,-0.351510,6.818566e-38


#### Why Feature Select?

One of the reasons feature selection is done, automatically or otherwise, is that there might be strong correlations between features. Also recall polynomial regression: a large number of features can lead to overfitting. Feature selection helps curb the problem of the curse of dimensionality, where centrality measures often used in statistics go wonky at higher dimensions. Between feature-engineering which we did some of, earlier, and feature selection, is where a lot of smarts and domain knowledge comes in. You will gain this with experience.

### Create a pipeline to feature-select, standardize and train!

We shall use sklearn pipelines to do correlation-with-response based feature selection for our SVM model. Maybe such feature-selection will improve the abysmal performance. 

This does not reduce the collinearity amongst the features, for which one either needs PCA, ICA, or some feature selection using the forward-backward algorithm. We do not have the time to approach it here. 

Its very important to do response based feature selection in the right way. If you remember, we separately standardized the training and test sets. This was to prevent **any** information about the overall mean and standard deviation leaking into the test set. 

But we played a bit loose with the rules there. We standardized on the entire training set. Instead we should have been standardizing separately in each cross-validation fold. There the original training set would be broken up into a sub-training and validation set, the standardization needed to be done on those separately. This can be implemented with `sklearn` pipelines.

Such kind of "data snooping" is relatively benign though, as it used no information about the response variable. But if you do any feature selection which uses the response variable, such as choosing the "k" most correlated variables from above, its not benign any more. This is because you have leaked the response from the validation into your sub-training set, and cannot thus be confident about your predictions: you might overfit. In such a situation, you must do the feature selection inside the cross-validation fold. See http://nbviewer.ipython.org/github/cs109/content/blob/master/lec_10_cross_val.ipynb from the 2013 course for a particularly dastardly case of this, where you see that the problem is particularly exacerbated when you have many more features than samples.

Lets do this here using sklearn pipelines.

In [68]:
from sklearn import feature_selection
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest

Lets define a scorer which returns the absolute values of the pearson correlation between the feature and the response for each sample. The specific form of the scorer is dictated to us in the API docs for `SelectKBest`, see [here](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html): the first argument must be an array of scores, and the second an array of p-values.

In [69]:
def pearson_scorer(X,y):
    rs=np.zeros(X.shape[1])
    pvals=np.zeros(X.shape[1])
    i=0
    for v in X.T:
        rs[i], pvals[i]=pearsonr(v, y)
        i=i+1
    return np.abs(rs), pvals    

Lets apply the feature selection to a model which did not have any automatic feature selection and performed rather poorly before: the linear SVM. 

The `Pipeline` feature of sklearn chains various parts of a machine learning algorithm together. In this case we want to chain feature-selection and training in such a way that both happen freshly for each cross-validation fold (we wont bother to standardize in each cross-validation fold separately here for brevity, although you might want to do this).
We use the `SelectKBest` meta estimator to select the 25 most correlated/anti-correlated features. We create an instance of this meta-estimator, `selectorlinearsvm`. We then combine it with the linear SVC estimators into the pipeline `pipelinearsvm`: the `Pipeline` function simply takes a list of `scikit-learn` estimators and wraps them together into a new estimator object, which can then be passed to `GridSearchCV` via our `do_classify` function. Notice how this new estimator object can be used exactly the same way as a single classifier can be used in `scikit-learn`..this uniformity of interface is one of the nice features of `sklearn`!

In [70]:
selectorlinearsvm = SelectKBest(k=25, score_func=pearson_scorer)
pipelinearsvm = Pipeline([('select', selectorlinearsvm), ('svm', LinearSVC(loss="hinge"))])


#### Let us run the pipelined classifier 

We'll run the classifier and compare the results using the ROC curve to the previous SVM result.

In [71]:
pipelinearsvm, _,_,_,_  = do_classify(pipelinearsvm, {"svm__C": [0.00001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split)

############# based on standard predict ################
Accuracy on training data: 0.70
Accuracy on test data:     0.70
########################################################


What features did the pipelined classifier use? We can access them so:

In [72]:
np.array(lcols)[pipelinearsvm.get_params()['select'].get_support()]

array(['bb_pct', 'sar_signal', 'plus_di', 'minus_di', 'adx_direction',
       'cci', 'macd', 'macd_hist', 'macd_signal', 'mfi', 'roc', 'rsi',
       'ult_osc', 'willr', 'wr_signal', 'ad_osc', 'ad_signal',
       'stoch_slowk', 'stoch_slowd', 'sslow_signal', 'stoch_fastk',
       'stoch_fastd', 'srsi_signal', 'sr_res1', 'sr_res3'], 
      dtype='|S13')

We plot the ROC curves, using the label `svm-feature-selected` for the pipelined classifier `pipelinearsvm`. We plot it alongside the older logistic-with lasso and all-features SVM for comparison

In [73]:
evaluate(pipelinearsvm)

ROI baseline: -3.47%
ROI "result" buy-only: 50.29%
ROI "result" buy-sell: 130.60%
ROI "pred" buy-only: -5.63%
ROI "pred" buy-sell: -8.34%


### Balancing train set to test set for training.

In [74]:
jtrain=np.arange(0, ytrain.shape[0])
n_pos=len(jtrain[ytrain==1])
n_neg=len(jtrain[ytrain==0])
print n_pos, n_neg

569 689


There are many more negative samples in the training set. We want to balance the negative samples to the positive samples. So lets sample $n_{+}$ samples from the negative samples in the training set (without replacement).

In [75]:
ineg = np.random.choice(jtrain[ytrain==0], n_pos, replace=False)

We concatenate all the indexes and use them to select a new training set from the old one.

In [76]:
alli=np.concatenate((jtrain[ytrain==1], ineg))
alli.shape

(1138,)

In [77]:
Xtrain_new = Xtrain[alli]
ytrain_new = ytrain[alli]
Xtrain_new.shape, ytrain_new.shape

((1138, 70), (1138,))

We store these into a new split variable `reuse_split_new`.

In [78]:
reuse_split_new=dict(Xtrain=Xtrain_new, Xtest=Xtest, ytrain=ytrain_new, ytest=ytest)

Note that the test sets are identical as before. This is as, even though we are training the SVM classifier in the "naturally" unfound situation of balanced classes, we **must test it in the real-world scenario of imbalance**.

#### 4.2 Train a linear SVM on this balanced set

Train a non-feature-selected linear SVM on this new balanced set as a comparison to both our old SVM on the imbalanced data set `clfsvm` and the feature-selected linear SVM `pipelinearsvm`. Store this new classifier in the variable `clfsvm_b`.

Compare the performances of all three of these classifiers using the roc curve plot, with the new `clfsvm_b` labeled as `svm-all-features-balanced`. 

In [79]:
#your code here
clfsvm_b = Pipeline([('select', selectorlinearsvm), ('svm', LinearSVC(loss="hinge"))])
clfsvm_b, _,_,_,_  = do_classify(clfsvm_b, {"svm__C": [0.00001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)

############# based on standard predict ################
Accuracy on training data: 0.69
Accuracy on test data:     0.68
########################################################


In [80]:
clfsvm_b

Pipeline(steps=[('select', SelectKBest(k=25, score_func=<function pearson_scorer at 0x10b7a2398>)), ('svm', LinearSVC(C=0.001, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

#### 4.3 Implement a RBF based pipelined (feature-selected) classifier on the balanced set.

In [81]:
from sklearn.svm import SVC

In [82]:
%%time
selectorsvm2 = SelectKBest(k=25, score_func=pearson_scorer)
pipesvm2 = Pipeline([('select2', selectorsvm2), ('svm2', SVC())])
pipesvm2, _,_,_,_  = do_classify(pipesvm2, {"svm2__C": [1e8, 1e9, 1e10], "svm2__gamma": [ 1e-9, 1e-10, 1e-11]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)

############# based on standard predict ################
Accuracy on training data: 0.68
Accuracy on test data:     0.71
########################################################
CPU times: user 3.5 s, sys: 34.5 ms, total: 3.54 s
Wall time: 3.61 s


In [83]:
pipesvm2.get_params()

{'select2': SelectKBest(k=25, score_func=<function pearson_scorer at 0x10b7a2398>),
 'select2__k': 25,
 'select2__score_func': <function __main__.pearson_scorer>,
 'svm2': SVC(C=100000000.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
   gamma=1e-09, kernel='rbf', max_iter=-1, probability=False,
   random_state=None, shrinking=True, tol=0.001, verbose=False),
 'svm2__C': 100000000.0,
 'svm2__cache_size': 200,
 'svm2__class_weight': None,
 'svm2__coef0': 0.0,
 'svm2__degree': 3,
 'svm2__gamma': 1e-09,
 'svm2__kernel': 'rbf',
 'svm2__max_iter': -1,
 'svm2__probability': False,
 'svm2__random_state': None,
 'svm2__shrinking': True,
 'svm2__tol': 0.001,
 'svm2__verbose': False}

In [84]:
evaluate(pipesvm2)

ROI baseline: -3.47%
ROI "result" buy-only: 50.29%
ROI "result" buy-sell: 130.60%
ROI "pred" buy-only: -2.08%
ROI "pred" buy-sell: -1.24%


In [105]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

# selector of  features
featSelector = SelectKBest(k=20, score_func=pearson_scorer)

print "#############====================== Linear SVM ========================#############"
clfsvm_b = Pipeline([('select', featSelector), ('svm', LinearSVC(loss="hinge"))])
clfsvm_b, _,_,_,_  = do_classify(clfsvm_b, {"svm__C": [0.00001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(clfsvm_b)
print "#############====================== RBF SVC ===========================#############"
pipesvm2 = Pipeline([('select', featSelector), ('svm2', SVC())])
pipesvm2, _,_,_,_  = do_classify(pipesvm2, {"svm2__C": [1e8, 1e9, 1e10], "svm2__gamma": [ 1e-9, 1e-10, 1e-11]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipesvm2)
print "#############====================== Random Forest =====================#############"
pipeRF = Pipeline([('select', featSelector), ('RF', RandomForestClassifier())])
pipeRF, _,_,_,_  = do_classify(pipeRF, {"RF__max_depth": [3,5,7,10,15,25,50], "RF__n_estimators": [5,10,20,40],"RF__max_features": [1,2,3]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeRF)
print "#############====================== AdaBoost ==========================#############"
pipeAda = Pipeline([('select', featSelector), ('Ada', AdaBoostClassifier())])
pipeAda, _,_,_,_  = do_classify(pipeAda, {"Ada__n_estimators": [5,10,20,40],"Ada__learning_rate": [0.1,0.5,1.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeAda)
print "#############====================== Gaussian NB ==========================#############"
pipeNB = Pipeline([('select', featSelector), ('NB', GaussianNB())])
pipeNB, _,_,_,_  = do_classify(pipeNB, {}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeNB)

#############====================== Linear SVM ========================#############
Accuracy on training data: 0.69
Accuracy on test data:     0.68
########################################################
ROI baseline: -3.47%
ROI "result" buy-only: 50.29%
ROI "result" buy-sell: 130.60%
ROI "pred" buy-only: -7.02%
ROI "pred" buy-sell: -11.08%
#############====================== RBF SVC ===========================#############
Accuracy on training data: 0.67
Accuracy on test data:     0.67
########################################################
ROI baseline: -3.47%
ROI "result" buy-only: 50.29%
ROI "result" buy-sell: 130.60%
ROI "pred" buy-only: -5.75%
ROI "pred" buy-sell: -8.41%
#############====================== Random Forest =====================#############
Accuracy on training data: 0.72
Accuracy on test data:     0.65
########################################################
ROI baseline: -3.47%
ROI "result" buy-only: 50.29%
ROI "result" buy-sell: 130.60%
ROI "pred" buy-only: -1